## Regression

In [20]:
import pandas as pd
import numpy as np
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from sklearn.linear_model import LinearRegression

In [21]:
data = pd.read_csv(filepath_or_buffer='../../resources/intraday.csv', parse_dates=['time'], index_col='time')

In [22]:
data.head(10)

,Close
time,
2018-01-01 22:00:00+00:00,1.201205
2018-01-02 04:00:00+00:00,1.207055
2018-01-02 10:00:00+00:00,1.204440
2018-01-02 16:00:00+00:00,1.205800
2018-01-02 22:00:00+00:00,1.204690
2018-01-03 04:00:00+00:00,1.203825
2018-01-03 10:00:00+00:00,1.202355
2018-01-03 16:00:00+00:00,1.201445
2018-01-03 22:00:00+00:00,1.201450


In [23]:
data.tail(10)

,Close
time,
2019-12-26 16:00:00+00:00,1.109655
2019-12-26 22:00:00+00:00,1.111890
2019-12-27 04:00:00+00:00,1.113860
2019-12-27 10:00:00+00:00,1.116300
2019-12-27 16:00:00+00:00,1.117580
2019-12-29 22:00:00+00:00,1.119920
2019-12-30 04:00:00+00:00,1.119940
2019-12-30 10:00:00+00:00,1.120095
2019-12-30 16:00:00+00:00,1.119920


In [24]:
data.Close.to_frame()

,Close
time,
2018-01-01 22:00:00+00:00,1.201205
2018-01-02 04:00:00+00:00,1.207055
2018-01-02 10:00:00+00:00,1.204440
2018-01-02 16:00:00+00:00,1.205800
2018-01-02 22:00:00+00:00,1.204690
...,...
2019-12-29 22:00:00+00:00,1.119920
2019-12-30 04:00:00+00:00,1.119940
2019-12-30 10:00:00+00:00,1.120095


Getting more statistical insight into the dataset

In [25]:
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2070 entries, 2018-01-01 22:00:00+00:00 to 2019-12-30 22:00:00+00:00
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Close   2070 non-null   float64
dtypes: float64(1)
memory usage: 32.3 KB


In [26]:
data.describe()

,Close
count,2070.000000
mean,1.150400
std,0.041544
min,1.088635
25%,1.119714
50%,1.137698
75%,1.169170
max,1.253920


Since the only attribute we need is closing price
we modify the data frame


In [27]:
data = data.Close.to_frame()

In [28]:
data.head(10)

,Close
time,
2018-01-01 22:00:00+00:00,1.201205
2018-01-02 04:00:00+00:00,1.207055
2018-01-02 10:00:00+00:00,1.204440
2018-01-02 16:00:00+00:00,1.205800
2018-01-02 22:00:00+00:00,1.204690
2018-01-03 04:00:00+00:00,1.203825
2018-01-03 10:00:00+00:00,1.202355
2018-01-03 16:00:00+00:00,1.201445
2018-01-03 22:00:00+00:00,1.201450


Since the only attribute we need is closing price
we modify the data frame


In [29]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=data.index, y=data.Close, name='Close'))

fig.update_layout(title='EUR/USD', xaxis_title='Time', yaxis_title='Price')

fig.show()

In [30]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=data.loc['2018'].index, y=data.loc['2018'].Close, name='Close'))

fig.update_layout(title='EUR/USD', xaxis_title='Time', yaxis_title='Price')

fig.show()

We proceed to define baseline and compute return

In [31]:
data['returns'] = np.log(data.div(data.shift(1)))

In [32]:
data.head(10)

,Close,returns
time,,
2018-01-01 22:00:00+00:00,1.201205,NaN
2018-01-02 04:00:00+00:00,1.207055,0.004858
2018-01-02 10:00:00+00:00,1.204440,-0.002169
2018-01-02 16:00:00+00:00,1.205800,0.001129
2018-01-02 22:00:00+00:00,1.204690,-0.000921
2018-01-03 04:00:00+00:00,1.203825,-0.000718
2018-01-03 10:00:00+00:00,1.202355,-0.001222
2018-01-03 16:00:00+00:00,1.201445,-0.000757
2018-01-03 22:00:00+00:00,1.201450,0.000004


Predicting returns using linear regression

In [33]:
data['lag1'] = data.returns.shift(1)

In [34]:
data.dropna(inplace=True)

In [35]:
data

,Close,returns,lag1
time,,,
2018-01-02 10:00:00+00:00,1.204440,-0.002169,0.004858
2018-01-02 16:00:00+00:00,1.205800,0.001129,-0.002169
2018-01-02 22:00:00+00:00,1.204690,-0.000921,0.001129
2018-01-03 04:00:00+00:00,1.203825,-0.000718,-0.000921
2018-01-03 10:00:00+00:00,1.202355,-0.001222,-0.000718
...,...,...,...
2019-12-29 22:00:00+00:00,1.119920,0.002092,0.001146
2019-12-30 04:00:00+00:00,1.119940,0.000018,0.002092
2019-12-30 10:00:00+00:00,1.120095,0.000138,0.000018


In [36]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=data.returns, y=data.lag1, mode='markers'))

fig.update_layout(title='EUR/USD', xaxis_title='lag1', yaxis_title='Returns')

fig.show()
